# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [ ]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [ ]:
class EncoderDecoder(nn.Module):
    '''
    encoder-decoder 모델 구현
    '''
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
      super(EncoderDecoder, self).__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.src_embed = src_embed 
      self.tgt_embed = tgt_embed
      self.generator = generator
    
    
    def forward(self, src, tgt, src_mask, tgt_mask):
      '''
      masked src와 tgt sequence를 가져와서 처리
      인코더의 출력값을 메모리 입력값으로 받아서 순전파
      '''
      return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    

    def encode(self, src, src_mask):
      '''
      input(단어 index) 받아서 인코딩
      '''
      return self.encoder(self.src(self.src_embed(src), src_mask))
    
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
      '''
      ouput(index)와 인코더의 어텐션 출력을 받아서 디코딩
      '''
      return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    '''
    decoder의 output (n_batch×seq_len×d_model) matrix에서 다시 실제 token의 sentence로 변환해주는 generator
    '''
    def __init__(self, d_model, vocab):
      super(Generator, self).__init__()
      self.proj = nn.Linear(d_model, vocab) # FC layer거쳐 마지막 dimention을 d_model에서 len(vocab)으로 변경 -> voca에 대응 시키기 위해
    

    def forward(self, x):
      return F.log_softmax(self.proj(x), dim=-1) # softmax함수를 사용해 각 voca에 대한 확률값으로 변환 (dim=-1인 이유 : 마지막 dimention인 len(voca)에 대한 확률값 구하기 위해)

### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [ ]:
def clones(module, N):
  '''
  N개의 layer를 clone하여 만듭니다. 
  '''
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
    '''
    N개의 layer로 구성된 encoder
    '''
    def __init__(self, layer, N):
      super(Encoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size) # layer normalization 
    
    def forward(self, x, mask): ### encoder에 왜 mask가 들어가지?
      for layer in self.layers:
        x = layer(x,mask)
        return self.norm(x) # layer 마다 normalize 후 반환

In [ ]:
class LayerNorm(nn.Module):
    '''
    layer.size(features)를 받아서 normalize 해준다. 
    '''
    def __init__(self, features, eps=1e-6):
      super(LayerNorm, self).__init__()
      self.a_2 = nn.Parameter(torch.ones(features))
      self.b_2 = nn.Parameter(torch.zeros(features))
      self.eps = eps
    
    def forward(self, x):
      mean = x.mean(-1, keepdim=True)
      std = x.std(-1, keepdim=True)
      return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class SublayerConnection(nn.Module):
    '''
    residual connection : output을 그대로 사용하지 않고, output에 input을 추가적으로 더한 값을 사용함으로써 gradient vanishing 방지
    drop out : 오버 피팅 방지
    '''
    def __init__(self, size, dropout):
      super(SublayerConnection, self).__init__()
      self.norm = LayerNorm(size)
      self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
      return x + self.dropout(sublayer(self.norm(x))) # x(입력) + dropout한 sub layer(x)

In [ ]:
class EncoderLayer(nn.Module):
    '''
    Encoder는 self-attention과 feed forward, 총 2개의 sublayer로 이루어져 있다.
    '''
    def __init__(self, size, self_attn, feed_forward, dropout):
      super(EncoderLayer, self).__init__()
      self.self_attn = self_attn
      self.feed_forward = feed_forward
      self.sublayer = clones(SublayerConnection(size, dropout), 2)
      self.size = size
        
    def forward(self, x, mask):
      '''
      sublayer는 입력값 x뿐 아니라 적용할 layer도 입력으로 받아야 함.
      sublayer[0]은 self_attn, sublayer[1]은 feedforward를 적용 layer로 사용
      '''
      x = self.sublayer[0](x, lambda x: self.self_asttn(x, x, x, mask))
      return self.sublayer[1](x, self.feed_forward)

### Decoder
- `Decoder`
- `DecoderLayer`

In [ ]:
class Decoder(nn.Module):
    '''
    N개의 디코더 layer로 구성되어 있다.
    '''
    def __init__(self, layer, N):
      super(Decoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
      '''
      memory : encoder의 output, decoder layer의 두번째 self_attn에 입력한다. 
      '''
      for layer in self.layers:
        x = layer(x, memory, src_mask, tgt_mask)
      return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):
    '''
    self_attn, src_attn, feed_forward, 총 3개의 sub layer로 이루어져 있다. 
    '''
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
      super(DecoderLayer, self).__init__()
      self.size = size
      self.self_attn = self_attn
      self.src_attn = src_attn # 쿼리 : 디코더에서의 input / 키,밸류 : 인코더의 output
      self.feed_forward = feed_forward
      self.sublayer = clones(SublayerConnection(size, dropout), 3) # 3개의 layer
    
    
    def forward(self, x, memory, src_mask, tgt_mask):
      '''
      sublayer[0] : self_attn
      sublayer[1] : src_attn
      sublayer[2] : feed forward
      '''
      m = memory
      x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x,tgt_mask))
      x = self.sublayer[1](x, lambda x: self.src_attn(x,m,m,src_mask))
      return self.sublayer[2](x, self.feed_forward)

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
  d_k = query.size(-1)
  scores = torch.matmul(query, key.transpose(-2, -1)) \
          / math.sqrt(d_k)
  if mask is not None:
    scores = scores.masked_fill(mask == 0, -1e9)
  p_attn = F.softmax(scores, dim = -1)
  if dropout is not None:
    p_attn = dropout(p_attn)
  return torch.matmul(p_attn, value), p_attn

###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라
- score : (d_k, m)
- p_attn : (d_k, m)
- attention : (m, d_k)

### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : (12, 8, 1, 1)
- p_attn : (12, 8, 1, 1)
- attention : (12, 8, 1, 64)

- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [ ]:
class MultiHeadedAttention(nn.Module):
    '''
    h : head 갯수
    d_model : 모델 차원
    '''
    def __init__(self, h, d_model, dropout=0.1):
      super(MultiHeadedAttention, self).__init__()
      assert d_model % h == 0 # 입력 벡터를 h개로 나눠야 하므로 체크 필요
      self.d_k = d_model // h
      self.h = h
      self.linears = clones(nn.Linear(d_model, d_model), 4) # Linear module이 4개 
      self.attn = None
      self.dropout = nn.Dropout(p=dropout)
      
    def forward(self, query, key, value, mask=None):
      if mask is not None:
        # 마스크를 모든 head에 동일하게 적용
        mask = mask.unsqueeze(1) # 마스크 벡터의 head 부분에 해당하는 값을 1로 채움
      nbatches = query.size(0) # query size : (nbatches, length, d_model)

      # 1) Q, K, V에 linear projection을 취한 후 사이즈 변환 => h*d_k
      query, key, value = \
          [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
          for l, x in zip(self.linears, (query, key, value))]
      # size : (nbatches, h, length, d_k)

      # 2) (length, d_k) 사이즈의 벡터들에 attention을 h번 적용
      x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
      # x size : (nbatches, h, length, d_k)

      # 3) x를 이어 붙인 후 마지막 linear projection 적용
      x = x.transpose(1,2).contiguous().view(nbatches, -1, self.h*self.d_k)
      return self.linears[-1](x)
      # size : (nbatches, length, d_model)

### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : 64
- `nn.Linear(d_model, d_model)(query)` : (12, 152) 
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : (12, 1, 8, 64)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : (12, 8, 1, 64)

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [ ]:
class PositionwiseFeedForward(nn.Module):
    '''
    토큰 단위로 feedforward net(ffn)를 적용 
    d_model 차원의 입력값을 d_ff차원의 공간에 임베딩 -> ReLU함수를 취한 후, 다시 d_model 차원으로 임베딩
    '''
    def __init__(self, d_model, d_ff, dropout=0.1):
      super(PositionwiseFeedForward, self).__init__()
      self.w_1 = nn.Linear(d_model, d_ff)
      self.w_2 = nn.Linear(d_ff, d_model)
      self.dropout = nn.Dropout(dropout)

    def forward(self, x):
      return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [ ]:
class Embeddings(nn.Module):
  
    def __init__(self, d_model, vocab):
      super(Embeddings, self).__init__()
      self.lut = nn.Embedding(vocab, d_model)
      self.d_model = d_model
    
    def forward(self, x):
      return self.lut(x) * math.sqrt(self.d_model)

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [ ]:
class PositionalEncoding(nn.Module):
    '''
    input에 PE를 더해주는 모듈 구성
    '''
    def __init__(self, d_model, dropout, max_len = 5000):
      super(PositionalEncoding, self).__init__()
      self.dropout = nn.Dropout(p=dropout)

      pe = torch.zeros(max_len, d_model)
      position = torch.arange(0, max_len).unsqueeze(1) # (max_len, 1)
      div_term = torch.exp(torch.arange(0, d_model, 2)* -(math.log( 10000.0)/ d_model)) # (d_model/2)

      pe[:, 0::2] = torch.sin(position * div_term)
      pe[:, 1::2] = torch.cos(position * div_term)
      pe = pe.unsqueeze(0) # (1, max_len, d_model)
      self.register_buffer('pe', pe)

    def forward(self, x):
      x = x + Variable(self.pe[:, :x.size(1)],
                       requires_grad=False)
      return self.dropout(x)

### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : (512, 1)
- `div_term` : (256)
- `position * div_term` : (512, 256)

### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [ ]:
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    '''
    하이퍼 파라미터 값을 받고 모델 생성
    '''
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # Initialize parameters
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [ ]:
model = make_model(10,10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라




In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
encoder.layers.0.self_attn.linears.0.weight 	 torch.Size([512, 512])
encoder.layers.0.self_attn.linears.0.bias 	 torch.Size([512])
encoder.layers.0.self_attn.linears.1.weight 	 torch.Size([512, 512])
encoder.layers.0.self_attn.linears.1.bias 	 torch.Size([512])
encoder.layers.0.self_attn.linears.2.weight 	 torch.Size([512, 512])
encoder.layers.0.self_attn.linears.2.bias 	 torch.Size([512])
encoder.layers.0.self_attn.linears.3.weight 	 torch.Size([512, 512])
encoder.layers.0.self_attn.linears.3.bias 	 torch.Size([512])
encoder.layers.0.feed_forward.w_1.weight 	 torch.Size([2048, 512])
encoder.layers.0.feed_forward.w_1.bias 	 torch.Size([2048])
encoder.layers.0.feed_forward.w_2.weight 	 torch.Size([512, 2048])
encoder.layers.0.feed_forward.w_2.bias 	 torch.Size([512])
encoder.layers.0.sublayer.0.norm.a_2 	 torch.Size([512])
encoder.layers.0.sublayer.0.norm.b_2 	 torch.Size([512])
encoder.layers.0.sublayer.1.norm.a_2 	 torch.Size([512])
encoder.layers.0.sublayer.1.norm